In [1]:
!pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4

In [2]:
from datasets import list_datasets, load_dataset


In [3]:
imdb=load_dataset("imdb")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
imdb.keys()

dict_keys(['train', 'test', 'unsupervised'])

In [5]:
imdb["train"].column_names

['text', 'label']

In [6]:
imdb["train"][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [7]:
imdb["train"].shape

(25000, 2)

In [8]:
imdb["test"].shape

(25000, 2)

In [9]:
imdb["train"].features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['neg', 'pos'], id=None)}

In [10]:
!pip install transformers

In [11]:
from transformers import AutoTokenizer

In [12]:
tokenizer=AutoTokenizer.from_pretrained("distilbert-base-uncased")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [13]:
tokenizer.is_fast

True

In [14]:
type(tokenizer)

transformers.models.distilbert.tokenization_distilbert_fast.DistilBertTokenizerFast

In [15]:
tokenizer.model_max_length

512

In [16]:
tokenizer.vocab_size

30522

In [17]:
X_train=imdb["train"]["text"]
y_train=imdb["train"]["label"]
X_test=imdb["test"]["text"]
y_test=imdb["test"]["label"]

In [18]:
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

25000
25000
25000
25000


In [19]:
print(type(X_train))
print(type(y_train))


<class 'list'>
<class 'list'>


In [20]:
train_encodings=tokenizer(X_train, max_length=128, truncation=True, padding=True)
test_encodings=tokenizer(X_test, max_length=128, truncation=True, padding=True)

In [21]:
import tensorflow as tf

In [22]:
train_dataset=tf.data.Dataset.from_tensor_slices((dict(train_encodings), y_train))
test_dataset=tf.data.Dataset.from_tensor_slices((dict(test_encodings), y_test))

In [23]:
train_dataset=train_dataset.shuffle(len(X_train)).batch(16)
test_dataset=test_dataset.batch(16)

In [24]:
from transformers import TFAutoModelForSequenceClassification


In [25]:
from transformers import TFAutoModelForSequenceClassification
from transformers import create_optimizer
import tensorflow as tf

# Load the pre-trained model
model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# Define the optimizer using the transformers library
optimizer, _ = create_optimizer(
    init_lr=3e-5,
    num_train_steps=10000,  # Change this to the actual number of training steps
    num_warmup_steps=1000   # Change this to the actual number of warmup steps
)

# Define the loss and metrics
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = [tf.keras.metrics.SparseCategoricalAccuracy("accuracy")]

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

# Print the model summary
model.summary()


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_19 (Dropout)        multiple                  0 (unused)
                                                                 
Total params: 66955010 (255.41 MB)
Trainable params: 66955010 (255.41 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
# model=TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# opt=tf.keras.optimizers.Adam(learning_rate=3e-5)
# loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# metrics=tf.keras.metrics.SparseCategoricalAccuracy("accuracy")

# model.compile(optimizer=opt, loss=loss, metrics=metrics)
# model.summary()

In [27]:
model.fit(train_dataset, epochs=3)

Epoch 1/3


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1563/1563 [==============================] - 383s 213ms/step - loss: 0.3963 - accuracy: 0.8090
Epoch 2/3
1563/1563 [==============================] - 333s 213ms/step - loss: 0.2366 - accuracy: 0.9037
Epoch 3/3
1563/1563 [==============================] - 333s 213ms/step - loss: 0.1147 - accuracy: 0.9595


In [28]:
test_loss, test_acc=model.evaluate(test_dataset)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

1563/1563 [==============================] - 121s 74ms/step - loss: 0.3988 - accuracy: 0.8678
Test Loss: 0.3988
Test Accuracy: 0.8678
